In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Step 1: Prepare your dataset directory structure
data_directory = "EuroSAT_RGB/"
image_size = (128, 128)  # Adjust image size as needed

In [ ]:
# Step 2: Preprocess Data
def load_and_preprocess_data(data_dir, image_size):
    images = []
    labels = []

    i = 0
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            for filename in os.listdir(class_dir):
                if filename.lower().endswith((".tif", ".jpg", ".png")):
                    i += 1
                    if i == 25:
                      i = 0
                      img = cv2.imread(os.path.join(class_dir, filename), cv2.IMREAD_COLOR)

                      if img is not None:
                          # Reduce the number of channels to 3 (RGB)
                          img = img[:, :, :3]
                          img = cv2.resize(img, image_size)
                          img = img / 255.0  # Normalize pixel values to [0, 1]
                          images.append(img)
                          labels.append(class_name)

    return np.array(images), np.array(labels)

images, labels = load_and_preprocess_data(data_directory, image_size)

In [ ]:

# Encode labels and split the dataset
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
x_train, x_val, y_train, y_val = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

images, labels = load_and_preprocess_data(data_directory, image_size)

In [ ]:
# Step 3: Label encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

In [ ]:
# Step 4: Split the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
# Step 5: Build a CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_size + (3,)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Step 6: Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 7: Train the model
batch_size = 64
epochs = 40

datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(x_val, y_val))

In [ ]:
# Step 8: Evaluate the model
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print("Validation accuracy:", test_accuracy)

In [ ]:
# Step 9: Save the model
model.save("crop_classification_model.h5")